In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time as tm

In [2]:
# saving the main directory
main_directory = os.getcwd()
main_directory

'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forJuliana'

In [3]:
#Counting number of files on folder
list = os.listdir('files') 
number_files = len(list)
number_files

3

In [4]:
# Changing directory to the folder
os.chdir(r'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forJuliana\\files')
os.getcwd()


'C:\\Users\\greci\\Documents\\Jupiter_notebooks\\forJuliana\\files'

In [ ]:
#Using a loop to process all files and populate the main data-frame

# Starting process timming
start = tm.time()

# Creating main data-frame
appended_data = [] #This array will contain df as elements

i = 0
while i < number_files:
    
    ##############################################
    # Processing each file
    ##############################################
    # Saving file's name
    file = list[i]
    # Reading files
    ThreeCT_data = pd.ExcelFile(file)

    # Saving group's name
    group_name = file[0:8]


    # Reading sheets within files
    df1 = pd.read_excel(ThreeCT_data, '1', header=None)
    df2 = pd.read_excel(ThreeCT_data, '2', header=None)
    df3 = pd.read_excel(ThreeCT_data, '3', header=None)
    df4 = pd.read_excel(ThreeCT_data, '4', header=None)

    #Dropping the frist 13 rows and the row between the last reading and the "sum"
    rows_to_drop = np.arange(13)
    df1 = df1.drop(df1.index[rows_to_drop])
    df2 = df2.drop(df2.index[rows_to_drop])
    df3 = df3.drop(df3.index[rows_to_drop])
    df4 = df4.drop(df4.index[rows_to_drop])
    
    #locating empty and iso columns
    e1 = df1.loc[13].where(df1.loc[13].str.contains('empty')).dropna(how='all')
    e1.index #To getthe column value
    empty1 = e1.index[0]
    s1 = df1.loc[13].where(df1.loc[13].str.contains('iso')).dropna(how='all')
    s1.index #To getthe column value
    iso1 = s1.index[0]
    
    e2 = df2.loc[13].where(df2.loc[13].str.contains('empty')).dropna(how='all')
    e2.index #To getthe column value
    empty2 = e2.index[0]
    s2 = df2.loc[13].where(df2.loc[13].str.contains('iso')).dropna(how='all')
    s2.index #To getthe column value
    iso2 = s2.index[0]
    
    e3 = df3.loc[13].where(df3.loc[13].str.contains('empty')).dropna(how='all')
    e3.index #To getthe column value
    empty3 = e3.index[0]
    s3 = df3.loc[13].where(df3.loc[13].str.contains('iso')).dropna(how='all')
    s3.index #To getthe column value
    iso3 = s3.index[0]
    
    e4 = df4.loc[13].where(df4.loc[13].str.contains('empty')).dropna(how='all')
    e4.index #To getthe column value
    empty4 = e4.index[0]
    s4 = df4.loc[13].where(df4.loc[13].str.contains('iso')).dropna(how='all')
    s4.index #To getthe column value
    iso4 = s4.index[0]
    
    # Categorizing "duration" labels to iso or empty
    df1.iloc[1:2, empty1:empty1+1] = 'Duration_empty_(s)'
    df1.iloc[1:2, iso1:iso1+1] = 'Duration_iso_(s)'
    
    df2.iloc[1:2, empty2:empty2+1] = 'Duration_empty_(s)'
    df2.iloc[1:2, iso2:iso2+1] = 'Duration_iso_(s)'
    
    df3.iloc[1:2, empty3:empty3+1] = 'Duration_empty_(s)'
    df3.iloc[1:2, iso3:iso3+1] = 'Duration_iso_(s)'
    
    df4.iloc[1:2, empty4:empty4+1] = 'Duration_empty_(s)'
    df4.iloc[1:2, iso4:iso4+1] = 'Duration_iso_(s)'
    
    #Dropping 13th row, now the 0th row
    df1 = df1.drop(df1.index[0])
    df2 = df2.drop(df2.index[0])
    df3 = df3.drop(df3.index[0])
    df4 = df4.drop(df4.index[0])
    
    #Creating and assigning new header
    new_header = df1.iloc[0] #grab the first row for the header
    df1 = df1[1:] #take the data less the header row
    df1.columns = new_header #set the header row as the df header
    df2 = df2[1:] #take the data less the header row
    df2.columns = new_header #set the header row as the df header
    df3 = df3[1:] #take the data less the header row
    df3.columns = new_header #set the header row as the df header
    df4 = df4[1:] #take the data less the header row
    df4.columns = new_header #set the header row as the df header

    # Selecting columns of av verlocity and duration for iso and empty
    df1 = df1[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
    df1.columns=["AVG_VEL_PIXELS/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
    df1.reset_index(drop=True, inplace=True)

    df2 = df2[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
    df2.columns=["AVG_VEL_PIXELS/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
    df2.reset_index(drop=True, inplace=True)

    df3 = df3[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
    df3.columns=["AVG_VEL_PIXELS/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
    df3.reset_index(drop=True, inplace=True)

    df4 = df4[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
    df4.columns=["AVG_VEL_PIXELS/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
    df4.reset_index(drop=True, inplace=True)
    
    #Removing the Nan row between the last reading a the 'sum'
    df1 = df1.dropna(how='all')
    df2 = df2.dropna(how='all')
    df3 = df3.dropna(how='all')
    df4 = df4.dropna(how='all')

  
    # Creating time-column vector based on measurements' size
    unit = 20/(len(df1)-1)
    TIME_minutes = np.arange(unit,21,unit)
    TIME_minutes = np.append(TIME_minutes, "SUM")
    
      # Adding time in minutes' column
    df1.insert(0, 'TIME_STEP_minutes', TIME_minutes)
    df2.insert(0, 'TIME_STEP_minutes', TIME_minutes)
    df3.insert(0, 'TIME_STEP_minutes', TIME_minutes)
    df4.insert(0, 'TIME_STEP_minutes', TIME_minutes)

    #Adding mouse name
    df1.insert(0, 'MOUSE_NAME', [group_name + '_1']*5)
    df2.insert(0, 'MOUSE_NAME', [group_name + '_2']*5)
    df3.insert(0, 'MOUSE_NAME', [group_name + '_3']*5)
    df4.insert(0, 'MOUSE_NAME', [group_name + '_4']*5)

    #Summations a diffrences of duration (empty vs iso)
    sum1 = df1["DURATION_EMPTY_s"] + df1["DURATION_ISO_s"]
    sum2 = df2["DURATION_EMPTY_s"] + df2["DURATION_ISO_s"]
    sum3 = df3["DURATION_EMPTY_s"] + df3["DURATION_ISO_s"]
    sum4 = df4["DURATION_EMPTY_s"] + df4["DURATION_ISO_s"]
    
    diff1 = df1["DURATION_ISO_s"] - df1['DURATION_EMPTY_s']
    diff2 = df2["DURATION_ISO_s"] - df2['DURATION_EMPTY_s']
    diff3 = df3["DURATION_ISO_s"] - df3['DURATION_EMPTY_s']
    diff4 = df4["DURATION_ISO_s"] - df4['DURATION_EMPTY_s']
    
    # Changin any sum value from zero to 10^20
    sum1.loc[sum1 == 0] = pow(10,20)
    sum2.loc[sum2 == 0] = pow(10,20)
    sum3.loc[sum3 == 0] = pow(10,20)
    sum4.loc[sum4 == 0] = pow(10,20)
    
    # Calculating relative a normalized socialindexes
    df1['SOL_IDX_REL'] = 100 * diff1/sum1
    df2['SOL_IDX_REL'] = 100 * diff2/sum2
    df3['SOL_IDX_REL'] = 100 * diff3/sum3
    df4['SOL_IDX_REL'] = 100 * diff4/sum4
    
    df1['SOL_IDX_NORM'] = 100 * diff1/(5*60)
    df2['SOL_IDX_NORM'] = 100 * diff2/(5*60)
    df3['SOL_IDX_NORM'] = 100 * diff3/(5*60)
    df4['SOL_IDX_NORM'] = 100 * diff4/(5*60)
    
    # Changin any sum value from zero to 10^20
    

    appended_data.append(df1)
    appended_data.append(df2)
    appended_data.append(df3)
    appended_data.append(df4)

    ##########################################################################################################################
    

    i += 1
    

appended_data = pd.concat(appended_data)
end = tm.time()

# Priting running time and files processed
print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

# Exporting dt to csv
appended_data.to_excel('3CT_data.xlsx',index=False)


In [22]:
# Saving file's name
file = list[0]
# Reading files
ThreeCT_data = pd.ExcelFile(file)

# Saving group's name
group_name = file[0:8]


# Reading sheets within files
df1 = pd.read_excel(ThreeCT_data, '1', header=None)
rows_to_drop = np.arange(13)
#     rows_to_drop = np.append(rows_to_drop,[19])
df1 = df1.drop(df1.index[rows_to_drop])

#locating empty and iso columns
e1 = df1.loc[13].where(df1.loc[13].str.contains('empty')).dropna(how='all')
e1.index #To getthe column value
empty1 = e1.index[0]
s1 = df1.loc[13].where(df1.loc[13].str.contains('iso')).dropna(how='all')
s1.index #To getthe column value
iso1 = s1.index[0]

# Categorizing "duration" labels to iso or empty
df1.iloc[1:2, empty1:empty1+1] = 'Duration_empty_(s)'
df1.iloc[1:2, iso1:iso1+1] = 'Duration_iso_(s)'

    #Dropping 13th row, now the 0th row
df1 = df1.drop(df1.index[0])

    #Creating and assigning new header
new_header = df1.iloc[0] #grab the first row for the header
df1 = df1[1:] #take the data less the header row
df1.columns = new_header #set the header row as the df header

# Selecting columns of av verlocity and duration for iso and empty
df1 = df1[["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
df1.columns=["AVG_VEL_PIXELS/s)", "DURATION_EMPTY_s", "DURATION_ISO_s"]
df1.reset_index(drop=True, inplace=True)

df1 = df1.dropna(how='all')
df1

,AVG_VEL_PIXELS/s),DURATION_EMPTY_s,DURATION_ISO_s
0,40.2,82.3,209.3
1,27.2,99.5,185.8
2,28.7,119.8,163.8
3,25.4,91.6,183.3
5,30.4,393.2,742.2


In [23]:
len(df1)

5

In [38]:
# Creating time-column vector


# a = numpy.append(a, a[0])

# [5,10,15,20,"SUM"]
#     df1.insert(0, 'TIME_STEP_minutes', TIME_minutes)
TIME_minutes

array(['5.0', '10.0', '15.0', '20.0', 'SUM'], dtype='<U32')